In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
import json
import os
import sys
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from random import choice
from time import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df = pd.read_csv("datos_fotmob_completo.csv")
# Cambiamos los nombres de algunas variables, dropeamos los valores vacios y ordenamos por fecha
# La fecha va de la más antigua a la más nueva
df.rename(columns={'homeTeam_score': 'score_home',
          "awayTeam_score": "score_away"}, inplace=True)
df.rename(columns={'homeTeam': 'team_home',
          "awayTeam": "team_away"}, inplace=True)
df = df.sort_values(by="date", ignore_index=True)
df = df.drop(["discipline_away", "discipline_home", "duels_home", "duels_away",
             "defense_home", "defense_away", "shots_home", "shots_away"], axis=1)
df.head()

,id,date,leagueName,team_home,team_away,score_home,score_away,homeIdTeam,awayIdTeam,parentLeagueId,...,expected_goals_home,expected_goals_away,expected_goals_open_play_home,expected_goals_open_play_away,expected_goals_set_play_home,expected_goals_set_play_away,expected_goals_non_penalty_home,expected_goals_non_penalty_away,expected_goals_on_target_home,expected_goals_on_target_away
0,1474723,2014-01-01,Premier League,Swansea City,Manchester City,2,3,10003,8456,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1475995,2014-01-01,League Two,Scunthorpe United,Rochdale,3,0,8412,8493,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1474676,2014-01-01,Premier League,Arsenal,Cardiff City,2,0,9825,8344,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1474680,2014-01-01,Premier League,Crystal Palace,Norwich City,1,1,9826,9850,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1474689,2014-01-01,Premier League,Fulham,West Ham United,2,1,9879,8654,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
columns_to_split = [
    "accurate_passes_home", "accurate_passes_away", 
    "long_balls_accurate_home", "long_balls_accurate_away", 
    "accurate_crosses_home", "accurate_crosses_away",
    "tackles_succeeded_home", "tackles_succeeded_away", 
    "ground_duels_won_home", "ground_duels_won_away", 
    "aerials_won_home", "aerials_won_away",
    "dribbles_succeeded_home", "dribbles_succeeded_away"
]
# Dividir y procesar cada columna
for column in columns_to_split:
    # Dividir la columna en dos columnas separadas
    df[[column+'_numerico', column+'_porcentaje']] = df[column].str.split(' ', expand=True)
    # Eliminar los paréntesis y el símbolo de porcentaje de la columna de porcentaje
    df[column+'_porcentaje'] = df[column+'_porcentaje'].str.replace('(', '').str.replace(')', '').str.replace('%', '')
    # Convertir las columnas en tipos de datos numéricos
    df[column+'_numerico'] = pd.to_numeric(df[column+'_numerico'])
    df[column+'_porcentaje'] = pd.to_numeric(df[column+'_porcentaje'])
# Eliminar las columnas originales que han sido divididas
df.drop(columns=columns_to_split, inplace=True)

In [4]:
def get_features(data):
    return pd.DataFrame(data[5:]).T


def search_features(i, df, last_games):
    team_home = df.iloc[i]["team_home"]
    team_away = df.iloc[i]["team_away"]
    # Para hacer la busqueda más eficiente, solo buscamos en los partidos que estos equipos juegan
    df_match_home1 = df[df["team_home"] == team_home]
    df_match_home2 = df[df["team_away"] == team_home]
    df_match_away1 = df[df["team_home"] == team_away]
    df_match_away2 = df[df["team_away"] == team_away]

    merged_df_home = pd.concat([df_match_home1, df_match_home2])
    merged_df_home.drop_duplicates(inplace=True)
    merged_df_home = merged_df_home.sort_values(by="date", ascending=False)

    merged_df_away = pd.concat([df_match_away1, df_match_away2])
    merged_df_away.drop_duplicates(inplace=True)
    merged_df_away = merged_df_away.sort_values(by="date", ascending=False)

    # El index 0 es el partido, por lo que iteramos desde el 1
    data_home = pd.DataFrame()
    data_away = pd.DataFrame()
    try:
        for j in range(1, last_games+1):
            if team_home == merged_df_home.iloc[j]["team_home"]:
                d = get_features(merged_df_home.iloc[j])
                d["played_home"] = 1
                d["played_away"] = 0
                data_home = pd.concat([data_home, d])
            elif team_home == merged_df_home.iloc[j]["team_away"]:
                d = get_features(merged_df_home.iloc[j])
                d["played_home"] = 0
                d["played_away"] = 1
                data_home = pd.concat([data_home, d])

        for j in range(1, last_games+1):
            if team_away == merged_df_away.iloc[j]["team_home"]:
                d = get_features(merged_df_away.iloc[j])
                d["played_home"] = 1
                d["played_away"] = 0

                data_away = pd.concat([data_away, d])
            elif team_away == merged_df_away .iloc[j]["team_away"]:
                d = get_features(merged_df_away.iloc[j])
                d["played_home"] = 0
                d["played_away"] = 1
                data_away = pd.concat([data_away, d])

        data = pd.concat([data_home, data_away])

        data = data.reset_index(drop=True)

        data = data.stack().to_frame().T

        #### Hiperparametro de max score
        maxs = 4
        # Resultado:
        # W = home_win, D = draw, L = home_lose
        if df.iloc[i]["score_home"] > df.iloc[i]["score_away"]:
            data["Resultado"] = "W"
        elif df.iloc[i]["score_home"] == df.iloc[i]["score_away"]:
            data["Resultado"] = "D"
        elif df.iloc[i]["score_home"] < df.iloc[i]["score_away"]:
            data["Resultado"] = "L"

        # Resultado númerico:
        home_score = df.iloc[i]["score_home"]
        away_score = df.iloc[i]["score_away"]
        # Buscar a cual le queda
        if home_score >= maxs and away_score < maxs:
            data["Resultado_Numerico"] = f"{maxs}>-{away_score}"
        elif home_score < maxs and away_score >= maxs:
            data["Resultado_Numerico"] = f"{home_score}-{maxs}>"
        elif home_score >= maxs and away_score >= maxs:
            data["Resultado_Numerico"] = f"{maxs}>-{maxs}>"
        else:
            data["Resultado_Numerico"] = f"{home_score}-{away_score}"

        df_f = pd.DataFrame()
        for i in range(2*last_games):
            parte = data[i].add_suffix(f"_{i}",axis=1)
            df_f = pd.concat([df_f,parte],axis=1)
            
        df_f["Resultado"] = np.array(data["Resultado"])
        df_f["Resultado_Numerico"] = np.array(data["Resultado_Numerico"])

        return df_f
    except:
        return pd.DataFrame()
    



In [5]:
# # Filtros de liga y equipo
# ID GUARDADOS :
# Premier Inglaterra : 47, LaLiga España : 87, Serie A Italiana : 55, Bundesliga Alemania : 54, Ligue1 Francia : 53
df_test = df
id = [47, 87, 55, 54, 53]
df_test = df.loc[df['parentLeagueId'].isin(id)]
# Pruebas de dataframe


df_test = df_test[df_test["red_cards_away"].notnull()].reset_index(drop=True)

# Primero solo quiero
df_test = df_test.drop(["parentLeagueId", "leagueId",
                       "homeIdTeam", "awayIdTeam"], axis=1)
df_test = df_test.drop(["touches_opp_box_home", "touches_opp_box_away", "expected_goals_home", "expected_goals_away",
                       "expected_goals_open_play_home", "expected_goals_open_play_away", "expected_goals_set_play_home",
                        "expected_goals_set_play_away", "expected_goals_non_penalty_home", "expected_goals_non_penalty_away",
                          "expected_goals_on_target_home","expected_goals_on_target_away"], axis=1)
df_test = df_test.dropna(axis=0)
columnas = list(df_test.columns)
# columnas

In [ ]:
# Programa lineal
last_games = 15
resultados = []
df_nn = pd.DataFrame()
for i in tqdm(range(len(df_test))):
    info = search_features(i,df_test.head(i+1),last_games)
    resultados.append(info)
df_nn = pd.concat(resultados)

  0%|          | 0/92438 [00:00<?, ?it/s]

100%|██████████| 92438/92438 [1:43:25<00:00, 14.90it/s]  


In [6]:
# Programa paralelo (Cuidado que a veces las particiones se quedan sin memoria, error de pickle)

# Last games = Juegos pasados a tener en cuenta
# Num_trabajadores = Trabajadores en paralelo (usar -1 para todos los posibles)
# Se demora un par de minutos
last_games = 15
num_trabajadores = -1
resultados = Parallel(n_jobs=num_trabajadores,verbose=10)(
    delayed(search_features)(i,df_test.head(i+1), last_games) for i in range(len(df_test)))
df_nn = pd.concat(resultados)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.19696119375409696s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.12178421020507812s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.11834526062011719s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:    1.6s
[Parallel(n_job

In [7]:
df_nn.head()

,score_home_0,score_away_0,BallPossesion_home_0,BallPossesion_away_0,ShotsOnTarget_home_0,ShotsOnTarget_away_0,ShotsOffTarget_home_0,ShotsOffTarget_away_0,fouls_home_0,fouls_away_0,...,aerials_won_away_numerico_29,aerials_won_away_porcentaje_29,dribbles_succeeded_home_numerico_29,dribbles_succeeded_home_porcentaje_29,dribbles_succeeded_away_numerico_29,dribbles_succeeded_away_porcentaje_29,played_home_29,played_away_29,Resultado,Resultado_Numerico
0,2,1,36.0,64.0,4.0,3.0,8.0,2.0,9.0,15.0,...,18.0,44.0,9.0,47.0,12.0,63.0,0,1,W,2-0
0,1,2,41.0,59.0,3.0,6.0,4.0,3.0,11.0,9.0,...,11.0,42.0,13.0,57.0,12.0,48.0,0,1,D,0-0
0,0,0,65.0,35.0,3.0,1.0,3.0,5.0,14.0,20.0,...,9.0,43.0,14.0,64.0,9.0,60.0,0,1,W,2-1
0,1,1,73.0,27.0,8.0,2.0,5.0,1.0,7.0,9.0,...,27.0,61.0,10.0,59.0,10.0,48.0,0,1,D,1-1
0,3,0,55.0,45.0,6.0,0.0,6.0,3.0,6.0,12.0,...,33.0,65.0,7.0,39.0,6.0,60.0,1,0,W,4>-0


In [8]:
df_nn.to_csv("Dataframe_Auxiliar.csv")

In [9]:
df = pd.read_csv("Dataframe_Auxiliar.csv")
df = df.drop("Unnamed: 0", axis=1 )

In [10]:
df

,score_home_0,score_away_0,BallPossesion_home_0,BallPossesion_away_0,ShotsOnTarget_home_0,ShotsOnTarget_away_0,ShotsOffTarget_home_0,ShotsOffTarget_away_0,fouls_home_0,fouls_away_0,...,aerials_won_away_numerico_29,aerials_won_away_porcentaje_29,dribbles_succeeded_home_numerico_29,dribbles_succeeded_home_porcentaje_29,dribbles_succeeded_away_numerico_29,dribbles_succeeded_away_porcentaje_29,played_home_29,played_away_29,Resultado,Resultado_Numerico
0,2,1,36.0,64.0,4.0,3.0,8.0,2.0,9.0,15.0,...,18.0,44.0,9.0,47.0,12.0,63.0,0,1,W,2-0
1,1,2,41.0,59.0,3.0,6.0,4.0,3.0,11.0,9.0,...,11.0,42.0,13.0,57.0,12.0,48.0,0,1,D,0-0
2,0,0,65.0,35.0,3.0,1.0,3.0,5.0,14.0,20.0,...,9.0,43.0,14.0,64.0,9.0,60.0,0,1,W,2-1
3,1,1,73.0,27.0,8.0,2.0,5.0,1.0,7.0,9.0,...,27.0,61.0,10.0,59.0,10.0,48.0,0,1,D,1-1
4,3,0,55.0,45.0,6.0,0.0,6.0,3.0,6.0,12.0,...,33.0,65.0,7.0,39.0,6.0,60.0,1,0,W,4>-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11896,3,2,46.0,54.0,7.0,4.0,3.0,2.0,10.0,10.0,...,11.0,48.0,10.0,53.0,2.0,22.0,1,0,W,3-2
11897,1,3,62.0,38.0,7.0,6.0,6.0,6.0,10.0,11.0,...,5.0,83.0,22.0,71.0,6.0,38.0,1,0,W,2-1
11898,2,1,64.0,36.0,4.0,5.0,3.0,3.0,13.0,16.0,...,15.0,39.0,4.0,22.0,8.0,38.0,1,0,W,3-1
11899,4,2,45.0,55.0,9.0,3.0,3.0,8.0,18.0,6.0,...,15.0,39.0,4.0,22.0,8.0,38.0,0,1,W,3-1


In [11]:
last_games = 15


# Función para filtrar strings que contienen "home"
def contiene_home(string):
  return "home" in string.lower()

# Obtener strings que contienen "home"
features = list(filter(contiene_home, columnas))


# Medias_moviles, ARREGLAR NO TOMA SI ES VISITA O HOME
df_nn = pd.DataFrame()
for fea in features[1:]:
    for i in [5,10,15]:
        suma = 0
        for j in range(i):
          try:
            fea_away = fea.replace("home", "away")
            fea_not_home = fea.replace("home", "")
            df_nn[f"MediaMovil_{i}_{fea_not_home}_h_1"] = (df[f"{fea_away}_{j}"]*df[f"played_away_{j}"] + df[f"{fea}_{j}"]*df[f"played_home_{j}"])/i
            df_nn[f"MediaMovil_{i}_{fea_not_home}_h_2"] = (df[f"{fea_away}_{j}"]*(1-df[f"played_away_{j}"]) + df[f"{fea}_{j}"]*(1-df[f"played_home_{j}"]))/i

            df_nn[f"MediaMovil_{i}_{fea_not_home}_a_1"] = (df[f"{fea}_{j}"]*df[f"played_away_{j}"] + df[f"{fea_away}_{j}"]*df[f"played_home_{j}"])/i
            df_nn[f"MediaMovil_{i}_{fea_not_home}_a_2"] = (df[f"{fea}_{j}"]*(1-df[f"played_away_{j}"]) + df[f"{fea_away}_{j}"]*(1-df[f"played_home_{j}"]))/i
            
          except:
            print(fea)

df_nn["Resultado"] = np.array(df["Resultado"])
df_nn["Resultado_Numerico"] = np.array(df["Resultado_Numerico"])

C:\Users\gfuen\AppData\Local\Temp\ipykernel_16500\49544224.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_nn[f"MediaMovil_{i}_{fea_not_home}_h_1"] = (df[f"{fea_away}_{j}"]*df[f"played_away_{j}"] + df[f"{fea}_{j}"]*df[f"played_home_{j}"])/i
C:\Users\gfuen\AppData\Local\Temp\ipykernel_16500\49544224.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_nn[f"MediaMovil_{i}_{fea_not_home}_h_2"] = (df[f"{fea_away}_{j}"]*(1-df[f"played_away_{j}"]) + df[f"{fea}_{j}"]*(1-df[f"played_home_{j}"]))/i
C:\Users\gfuen\AppData\Local

In [12]:
df_nn.to_csv("Media_Movil_5.csv")

In [15]:
features

['team_home',
 'score_home',
 'BallPossesion_home',
 'ShotsOnTarget_home',
 'ShotsOffTarget_home',
 'fouls_home',
 'Offsides_home',
 'corners_home',
 'total_shots_home',
 'big_chance_home',
 'big_chance_missed_title_home',
 'blocked_shots_home',
 'shots_woodwork_home',
 'shots_inside_box_home',
 'shots_outside_box_home',
 'passes_home',
 'own_half_passes_home',
 'opposition_half_passes_home',
 'player_throws_home',
 'interceptions_home',
 'shot_blocks_home',
 'clearances_home',
 'keeper_saves_home',
 'duel_won_home',
 'yellow_cards_home',
 'red_cards_home',
 'accurate_passes_home_numerico',
 'accurate_passes_home_porcentaje',
 'long_balls_accurate_home_numerico',
 'long_balls_accurate_home_porcentaje',
 'accurate_crosses_home_numerico',
 'accurate_crosses_home_porcentaje',
 'tackles_succeeded_home_numerico',
 'tackles_succeeded_home_porcentaje',
 'ground_duels_won_home_numerico',
 'ground_duels_won_home_porcentaje',
 'aerials_won_home_numerico',
 'aerials_won_home_porcentaje',
 'dribbl